In [ ]:
import iar_project.utils as utils
import iar_project.features_extraction as extraction
import iar_project.test_clustering as clustering
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import cv2

In [ ]:
# How many images ?
nb_im = 12 #to which image
k = 0 # from which image
seg_img=utils.import_seg_results(k,nb_im)
sol_img=utils.import_solution()
sol_img=[sol_img]

In [ ]:
from skimage.filters.rank import entropy
from skimage.morphology import disk
features=extraction.get_features_color(seg_img)
img_5=seg_img[5]
img_5_gray=[cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in img_5]
img_5_entropy=[entropy(gray_im, disk(2)) for gray_im in img_5_gray ]

In [ ]:
import glob
import os
GENERAL_PATH = (
    "/Users/begue/OneDrive/Documents/EPFL/Master/Cours/MA2/IAPR/projet/iar_project"
)
DATA_PATH = GENERAL_PATH + "/data_project"
DATA_PATH2 = GENERAL_PATH + "/data_project2"
SEGMENTATION_OUTPUT_PATH = DATA_PATH + "/segmentation_results"
SEGMENTATION_OUTPUT_PATH2 = DATA_PATH2 + "/segmentation_results"
SOL_OUTPUT_PATH2 = DATA_PATH2 + "/train2_solutions"
CLUSTERING_OUTPUT_PATH2 = DATA_PATH2 + "/clustering_results"
img_list = []

seg_path = SEGMENTATION_OUTPUT_PATH2 + "/" + "*.png"
tmp = []
for filename in glob.glob(seg_path):
    if os.path.exists(filename):
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_list.append(img[10:118,10:118,:])
    else:
        print(f"Image {filename} not found.")
        print(filename)


In [ ]:
img_gray=[cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) for im in img_list]
img_entropy=[entropy(gray_im, disk(2)) for gray_im in img_gray ]
img_s=[cv2.cvtColor(im, cv2.COLOR_RGB2HSV)[:,:,0] for im in img_list]
img_g=[im[:,:,1] for im in img_list]



In [ ]:
features=extraction.get_features_color([img_list])
features = features[0]
print(features.shape)

In [ ]:
mean_ft = np.mean(features, axis=0)
std_ft = np.std(features, axis=0)
ft_norm = (features - mean_ft) / std_ft

In [ ]:
K = 25
kmeans = KMeans(n_clusters=K, n_init=50, init="random").fit(ft_norm[:,0:6])
labels = kmeans.labels_
unique_lab = np.unique(labels)
# nb_tiles = []
clu = []
for l, lab in enumerate(unique_lab):
    # print(lab)
    idx = np.argwhere(labels == lab)
    puzzle = []
    for i, id in enumerate(idx):
        # print(id[0])
        puzzle.append(img_list[id[0]])
    clu.append(puzzle)

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
for c,cluster in enumerate(clu):
    for t,tile in enumerate(cluster):
        ab = AnnotationBbox(OffsetImage(tile, zoom=.1), (t+1,c+1),frameon=False)
        ax.add_artist(ab)
    
plt.xticks(range(50))
plt.yticks(range(K+2))
#plt.ylim([0,4])
#plt.xlim([0,28])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))
for i in range(len(img_entropy)):
    ab = AnnotationBbox(OffsetImage(img_list[i], zoom=.1), (np.mean(img_g[i].ravel()), np.median(img_entropy[i].ravel())),frameon=False)
    ax.add_artist(ab)
    
plt.xticks(range(len(img_entropy)))
plt.yticks(range(len(img_entropy)))
plt.ylim([0,4])
plt.xlim([0,255])
plt.show()

## Image 5
Good separation using mean or std+mean of the entropiy with radius of 2

In [ ]:
plt.figure(figsize=(10,5))
for i in range(len(img_5)):
    plt.subplot(3,10,i+1)
    plt.imshow(img_5[i])
    plt.axis('off')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(len(img_5_entropy)):
    plt.subplot(3,10,i+1)
    plt.hist(img_5_entropy[i].ravel())
    plt.axis('off')
plt.tight_layout()

In [ ]:

fig, ax = plt.subplots()
for i in range(len(img_5_entropy)):
    ab = AnnotationBbox(OffsetImage(img_5[i], zoom=.1), (i, np.median(img_5_entropy[i].ravel())),frameon=False)
    ax.add_artist(ab)
    
plt.xticks(range(len(img_5_entropy)))
plt.yticks(range(len(img_5_entropy)))
plt.ylim([0,4])
plt.xlim([0,28])
plt.show()

In [ ]:
from skimage import feature
img_5=seg_img[5]
img_5_gray=[cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)[10:118,10:118] for im in img_5]
img_5_h=[np.fft.rfft2(im) for im in img_5_gray]
img_5_entropy=[entropy(gray_im, disk(2)) for gray_im in img_5_gray ]

img_5_c=[cv2.cvtColor(im, cv2.COLOR_RGB2HSV)[10:118,10:118,1] for im in img_5]

In [ ]:

fig, ax = plt.subplots()
for i in range(len(img_5_entropy)):
    ax.scatter(np.std(img_5_c[i].ravel()), np.median(img_5_entropy[i].ravel()))
    ab = AnnotationBbox(OffsetImage(img_5[i], zoom=.1), (np.std(img_5_c[i].ravel()), np.median(img_5_entropy[i].ravel())),frameon=False)
    ax.add_artist(ab)
    
#plt.xticks(range(len(img_5_entropy)))
#plt.yticks(range(len(img_5_entropy)))
#plt.ylim([0,4])
#plt.xlim([0,2])
plt.show()

## Image 6

Good separation using mean +std of saturation

In [ ]:
img_6=seg_img[6]
img_6_gray=[cv2.cvtColor(im, cv2.COLOR_RGB2HSV)[10:118, 10:118,1] for im in img_6]
img_6_h=[im[10:118, 10:118,0] for im in img_6]
img_6_entropy=[entropy(gray_im, disk(1)) for gray_im in img_6_gray ]

In [ ]:
plt.figure(figsize=(10,5))
for i in range(len(img_6)):
    plt.subplot(3,10,i+1)
    plt.imshow(img_6_gray[i])
    plt.axis('off')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(30,5))
for i in range(len(img_6_entropy)):
    plt.subplot(3,10,i+1)
    plt.hist(img_6_gray[i].ravel())
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
for i in range(len(img_6_gray)):
    ax.scatter(i, np.mean(img_6_gray[i].ravel()))
    ab = AnnotationBbox(OffsetImage(img_6[i], zoom=.1), (i, np.mean(img_6_gray[i].ravel())),frameon=False)
    ax.add_artist(ab)

plt.show()

In [ ]:
fig, ax = plt.subplots()
for i in range(len(img_6_gray)):
    ax.scatter(np.std(img_6_h[i].ravel()), np.mean(img_6_gray[i].ravel()))
    ab = AnnotationBbox(OffsetImage(img_6[i], zoom=.1), (np.std(img_6_h[i].ravel()), np.mean(img_6_gray[i].ravel())),frameon=False)
    ax.add_artist(ab)

plt.show()